In [1]:
import wget
import pandas as pd
import numpy as np
from Bio import Entrez
import Bio
import gzip
import sys
import csv
import math
import os

In [2]:
#Step1 transform gz file to csv file
def download_extract(url):
        #os.chdir(r"C:\Users\D.M\Desktop\pipeline_CGN\data")
        filename = wget.detect_filename(url)
        wget.download(url)
        with gzip.open(filename, 'r') as f1, open('data/gene2.txt', 'w') as f2:
            con = f1.readlines()
            for line in con:
# For the content read from the gz file, you should use decode() to convert it, otherwise it will report an error：TypeError: write() argument must be str, not bytes
                f2.write(line.decode('utf-8') )
        
#Convert space-separated plain text files to comma-separated files with the same name
        with open('data/all_PubMed.csv', 'w+',newline='') as csvfile:
             spamwriter = csv.writer(csvfile, dialect='excel')
             with open('data/gene2.txt', 'r',encoding='utf-8') as filein:
                for line in filein:
                    line_list = line.strip('\n').split('\t')
                    spamwriter.writerow(line_list)

In [3]:
#Step2 Search papers with cancer as a topic
def download_relatedpaper():
    Entrez.email = "571147054@qq.com"     # Always tell NCBI who you are
    handle = Entrez.einfo()
    result = handle.read()
    handle = Entrez.esearch(db='pubmed',term='cancer',mindate='1990/01/01',maxdate='2022/04/01',retmode='txt',retmax='10000000')
    record = Entrez.read(handle)
    txt = record["IdList"]
    f = open (r'data/cancer6_1.txt','w')
    print (txt,file = f)
    f.close()

In [4]:
#Step3 filter the cancer paper pubmed and gene ID from the all data
def filter_humangenepaper(dataset):
        df = pd.read_csv(dataset,sep=',+\s',engine='python')
        df.T.to_csv('data/cancer6_1转置.csv')
        df1 = pd.read_csv('data/cancer6_1转置.csv')
        #df1
        df2 = df1.applymap(lambda x: x.replace('[',''))
        df3 = df2.applymap(lambda x: x.replace(']',''))
        df4 = df3.applymap(lambda x: x.replace("'",""))
        df4.columns = ['PubMed_ID']
        df4.to_csv('data/cancer6_1_final.csv',index=0)
        df5 = pd.read_csv('data/cancer6_1_final.csv')
        #df5
        #filter cancer paper with pubmedID
        df6 = pd.read_csv('data/all_PubMed.csv')
        df7 = pd.merge(df6,df5,on='PubMed_ID')
        df8 = df7.to_csv('data/filiting cancer with geneID.csv')
        df8 = pd.read_csv('data/filiting cancer with geneID.csv')
        df9 = df7.loc[df7['#tax_id']==9606]
        df9.to_csv('data/filtered 9606.csv',index=0)

In [2]:
#permutation
def get_source_target(data, step, select_data, target_num):
#If a set was completed , output the current set
    if len(select_data) == target_num:
        print(select_data)
        return
    if step >= len(data):
        return
#Add the selected elements to the temporary output list,output a group of n 
    select_data.append(data[step])
    get_source_target(data, step + 1, select_data, target_num)
#To construct a new combination, first delete the elements to be excluded from the last output group
    select_data.pop()
#Recursive upward adjustment of one element                        
    get_source_target(data, step + 1, select_data, target_num)

In [6]:
#Step4 get all edge list
def get_alledge_list(data):
    df10 = pd.read_csv(data)
    #del df10['Unnamed: 0']
    #del df10['Unnamed: 0.1']
    del df10['#tax_id']
    #df10
    cloum_name = 'PubMed_ID'
    cloum_name1 = 'GeneID'
    df11 = df10[df10.groupby('PubMed_ID').GeneID.transform('count') < 100]
    df11[cloum_name].value_counts()[:400000]
    df12 = df10[df10.groupby('PubMed_ID').GeneID.transform('count') < 2]
    df12[cloum_name].value_counts()[:400000]
    df13 = df12[cloum_name1].value_counts()[:400000]
    df13.to_csv('data/singlegene_paper.csv')
    #singlegenepaper_number = len(zz)
    #singlegenepaper_number
    list = df11[cloum_name].drop_duplicates() #去重
    list1 = list.values.tolist()
    #len(list1)
    group = df11.groupby('PubMed_ID',as_index=False)
    #group.count()
    df10[cloum_name].value_counts()[:400000] # wihich means that (283489-283161) papers involving with more than 100 genes
    len( df10[cloum_name].value_counts()[:400000])# all needed papers(283411)
    tmp = sys.stdout
    sys.stdout = open('data/cancer4_1_outfile.csv','w')
    for i in list:
        data = group.get_group(i)
        list2 = data['GeneID'].values.tolist()
        get_source_target(list2, 0, [], 2) 
    sys.stdout = tmp #恢复到默认映射关系
    df14 = pd.read_csv('data/cancer4_1_outfile.csv',header=None)
    df14.columns = ['source','target']
    df15 = df14.applymap(lambda x: x.replace('[',''))
    df16 = df15.applymap(lambda x: x.replace(']',''))
    df16.to_csv('data/cancer4_1_outfile_indexed.csv')
    df17 = pd.read_csv('data/cancer4_1_outfile_indexed.csv')
    del df17['Unnamed: 0']
    df18 = df17.groupby(df17.columns.tolist()).size().reset_index().rename(columns={0:'weight'})
    #df18
    df18.to_csv('data/cancer_weight.csv',index=0)

In [7]:
download_extract(url='https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2pubmed.gz')

100% [........................................................................] 57674654 / 57674654

In [8]:
download_relatedpaper()

In [9]:
filter_humangenepaper(dataset='data/cancer6_1.txt')

In [10]:
get_alledge_list(data='data/filtered 9606.csv')

In [2]:
df19 = pd.read_csv('data/cancer_weight.csv')
df19

,source,target,weight
0,1,2,1
1,1,12,1
2,1,183,1
3,1,197,1
4,1,335,1
...,...,...,...
668305,118568805,118568823,1
668306,118568817,118568823,1
668307,120807611,120807612,1
668308,120807611,120807613,1


In [3]:
    df10 = pd.read_csv('data/filtered 9606.csv')
    #del df10['Unnamed: 0']
    #del df10['Unnamed: 0.1']
    del df10['#tax_id']
    #df10
    cloum_name = 'PubMed_ID'
    cloum_name1 = 'GeneID'
    df11 = df10[df10.groupby('PubMed_ID').GeneID.transform('count') < 100]
    df11[cloum_name].value_counts()[:400000]
    df12 = df10[df10.groupby('PubMed_ID').GeneID.transform('count') < 2]
    df12[cloum_name].value_counts()[:400000]
    df13 = df12[cloum_name1].value_counts()[:400000]
    df13.to_csv('data/singlegene_paper.csv')
    #singlegenepaper_number = len(zz)
    #singlegenepaper_number
    list = df11[cloum_name].drop_duplicates() #去重
    list1 = list.values.tolist()

In [10]:
len(df11[cloum_name].value_counts()[:400000]) #all used paper

283081

In [9]:
len(df12[cloum_name].value_counts()[:400000]) # single gene paper

173449

In [12]:
df12[cloum_name1].value_counts()[:400000]

7157         3256
1956         2396
2064         1460
673          1250
3845         1094
             ... 
100113382       1
101929983       1
574477          1
5345            1
8813            1
Name: GeneID, Length: 13077, dtype: int64

In [15]:
df11[cloum_name].value_counts()[:400000]

7157         8397
1956         5120
672          3001
2064         2956
3845         2588
             ... 
139886          1
441432          1
100240709       1
553989          1
100129148       1
Name: GeneID, Length: 19431, dtype: int64

In [17]:
df10[cloum_name].value_counts()[:4000000] #all paper

12477932    19898
32296183     8303
27701403     6656
34079125     4404
25416956     4395
            ...  
1717141         1
32407961        1
24886180        1
32232361        1
30414845        1
Name: PubMed_ID, Length: 283411, dtype: int64

In [15]:
dd = df12[cloum_name1].value_counts()[:400000]

In [16]:
dd.to_csv('sgpaper_gene.csv')